In [1]:
import os
import pickle
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Financial indicators to forecast
financial_indicators = [
    '10_year_rate', 'share_price', 'unemployment_rate', 'PPI',
    'OECD_CLI_index', 'CSI_index', 'gdp_per_capita'
]

# Recession probability target columns to exclude from exog
recession_targets = [
    'recession_probability', '1_month_recession_probability',
    '3_month_recession_probability', '6_month_recession_probability'
]


def check_stationarity(series, series_name):
    result = adfuller(series.dropna())
    print(f'\n{series_name} Stationarity Test:')
    print(f'ADF Statistic: {result[0]:.6f}')
    print(f'p-value: {result[1]:.6f}')
    if result[1] <= 0.05:
        print("Series is stationary")
        return True
    else:
        print("Series is non-stationary")
        return False


def prepare_exog_data(train_df, test_df, target_indicator):
    print(f"\nPreparing exogenous variables for {target_indicator}...")
    features_to_exclude = ['date'] + recession_targets + [target_indicator]
    available_features = [c for c in train_df.columns if c not in features_to_exclude]
    print(f"Available exogenous features: {len(available_features)}")
    print(f"Features: {available_features[:10]}..." if len(available_features) > 10 else f"Features: {available_features}")
    return train_df[available_features].copy(), test_df[available_features].copy(), available_features


def clean_exog_data(train_exog, test_exog):
    print(f"Cleaning exogenous data...")
    print(f"  Initial shapes - Train: {train_exog.shape}, Test: {test_exog.shape}")
    train_exog = train_exog.fillna(method='ffill').fillna(method='bfill')
    test_exog = test_exog.fillna(method='ffill').fillna(method='bfill')
    train_exog = train_exog.replace([np.inf, -np.inf], np.nan).fillna(method='ffill').fillna(method='bfill')
    test_exog = test_exog.replace([np.inf, -np.inf], np.nan).fillna(method='ffill').fillna(method='bfill')
    varying_cols = [c for c in train_exog.columns if train_exog[c].nunique() > 1]
    train_exog = train_exog[varying_cols]
    test_exog = test_exog[varying_cols]
    test_exog = test_exog[train_exog.columns]
    print(f"  Final shapes - Train: {train_exog.shape}, Test: {test_exog.shape}")
    print(f"  Remaining missing values - Train: {train_exog.isnull().sum().sum()}, Test: {test_exog.isnull().sum().sum()}")
    return train_exog, test_exog


def auto_arima_with_exog(series, exog, max_p=3, max_d=2, max_q=3, verbose=True):
    best_aic = float('inf')
    best_order = None
    best_model = None
    tried_orders = []

    print(f"\nRunning ARIMA grid search...")
    print(f"Series length: {len(series)}, Exog shape: {exog.shape}")

    for p in range(max_p + 1):
        for d in range(max_d + 1):
            for q in range(max_q + 1):
                try:
                    if verbose:
                        print(f"  Trying ARIMA order: ({p},{d},{q})")
                    model = ARIMA(series, order=(p, d, q), exog=exog)
                    fitted = model.fit()
                    if verbose:
                        print(f"    Success: AIC = {fitted.aic:.2f}")
                    if fitted.aic < best_aic:
                        best_aic, best_order, best_model = fitted.aic, (p, d, q), fitted
                    tried_orders.append((p, d, q, fitted.aic))
                except Exception as e:
                    if verbose:
                        print(f"    Failed ARIMA({p},{d},{q}): {str(e)[:100]}")
                    continue

    print(f"\nBest model found: ARIMA{best_order} with AIC = {best_aic:.2f}")
    print(f"Total successful fits: {len(tried_orders)}")
    return best_model, best_order, best_aic


def forecast_indicator_with_exog(train_df, test_df, indicator, max_p=3, max_d=2, max_q=3):
    print(f"\n{'='*60}\nFORECASTING: {indicator}\n{'='*60}")

    if indicator not in train_df.columns or indicator not in test_df.columns:
        print(f"ERROR: {indicator} not in datasets")
        return None

    train_series = train_df[indicator].dropna()
    test_series = test_df[indicator].dropna()
    print(f"Series lengths - Train: {len(train_series)}, Test: {len(test_series)}")
    if len(train_series) < 10:
        print(f"ERROR: Insufficient training data for {indicator}")
        return None

    try:
        train_exog, test_exog, exog_features = prepare_exog_data(train_df, test_df, indicator)
        train_exog, test_exog = train_exog.loc[train_series.index], test_exog.loc[test_series.index]
        train_exog, test_exog = clean_exog_data(train_exog, test_exog)
        if train_exog.shape[1] == 0:
            print(f"WARNING: No exogenous variables for {indicator}")
            return None
    except Exception as e:
        print(f"ERROR preparing exogenous: {e}")
        return None

    is_stationary = check_stationarity(train_series, indicator)

    try:
        model, order, aic = auto_arima_with_exog(train_series, train_exog, max_p, max_d, max_q)
        if model is None:
            print(f"ERROR: No ARIMA model for {indicator}")
            return None

        steps = min(len(test_series), len(test_exog))
        print(f"  Forecasting {steps} steps ahead")
        forecast = model.forecast(steps=steps, exog=test_exog.iloc[:steps])
        ci = model.get_forecast(steps=steps, exog=test_exog.iloc[:steps]).conf_int()

        actual = test_series.iloc[:steps]
        mae = np.mean(np.abs(forecast - actual))
        rmse = np.sqrt(np.mean((forecast - actual) ** 2))

        # fix MAPE mask alignment
        actual_np, forecast_np = actual.to_numpy(), np.asarray(forecast)
        mask = actual_np != 0
        mape = np.mean(np.abs((actual_np[mask] - forecast_np[mask]) / actual_np[mask])) * 100 if mask.any() else np.inf

        print(f"\nAccuracy Metrics:\n  MAE: {mae:.4f}\n  RMSE: {rmse:.4f}\n  MAPE: {mape:.2f}%")

        # ensure save folder
        os.makedirs("arima_models", exist_ok=True)
        with open(f"arima_models/{indicator}_arima_model.pkl", "wb") as f:
            pickle.dump(model, f)
        print(f"Model saved to arima_models/{indicator}_arima_model.pkl")

        return {
            'model': model, 'order': order, 'aic': aic,
            'forecast': forecast, 'forecast_lower': ci.iloc[:, 0],
            'forecast_upper': ci.iloc[:, 1], 'actual': actual,
            'mae': mae, 'rmse': rmse, 'mape': mape,
            'is_stationary': is_stationary, 'exog_features': exog_features,
            'train_length': len(train_series), 'test_length': len(actual)
        }

    except Exception as e:
        print(f"ERROR fitting ARIMA for {indicator}: {e}")
        return None


def plot_forecast_results(results, indicator, train_df, test_df):
    if results is None:
        return
    plt.figure(figsize=(15, 8))
    train_series = train_df[indicator].dropna()
    train_plot = train_series.tail(min(50, len(train_series)))
    train_dates = train_df.loc[train_plot.index, 'date']
    plt.plot(train_dates, train_plot.values, label='Training', color='blue', alpha=0.7)

    actual = results['actual']
    test_dates = test_df.loc[actual.index, 'date'].iloc[:len(actual)]
    plt.plot(test_dates, actual.values, label='Actual', color='green', linewidth=2, marker='o', markersize=4)

    forecast = results['forecast']
    forecast_dates = test_df.loc[actual.index, 'date'].iloc[:len(forecast)]
    plt.plot(forecast_dates, forecast, label='Forecast', color='red', linewidth=2, marker='s', markersize=4)

    plt.fill_between(forecast_dates, results['forecast_lower'], results['forecast_upper'],
                     alpha=0.3, color='red', label='95% CI')
    plt.title(f'{indicator} - ARIMA{results["order"]}\n'
              f'MAE:{results["mae"]:.4f} RMSE:{results["rmse"]:.4f} MAPE:{results["mape"]:.2f}%',
              fontsize=14)
    plt.xlabel('Date'); plt.ylabel('Value'); plt.legend(); plt.grid(alpha=0.3); plt.xticks(rotation=45)
    if len(train_dates) and len(test_dates):
        plt.axvline(x=test_dates.iloc[0], color='gray', linestyle='--', alpha=0.7)
    plt.tight_layout(); plt.show()


def run_time_series_forecasting_selected(train_selected, test_selected, financial_indicators=None, plot_results=True):
    print("=" * 80); print("TIME SERIES FORECASTING WITH SELECTED FEATURES"); print("=" * 80)
    if financial_indicators is None:
        financial_indicators = [
            '6_months_rate', '3_months_rate'
        ]
        # financial_indicators = [
        #     '10_year_rate', 'share_price', 'unemployment_rate', 'PPI',
        #     'OECD_CLI_index', 'CSI_index', 'gdp_per_capita'
        # ]
    train_df, test_df = train_selected.copy(), test_selected.copy()
    train_df['date'], test_df['date'] = pd.to_datetime(train_df['date']), pd.to_datetime(test_df['date'])

    available = [i for i in financial_indicators if i in train_df.columns and i in test_df.columns]
    print(f"Indicators to forecast: {available}")

    all_results, forecasted = {}, pd.DataFrame({'date': test_df['date'].copy()})
    for ind in available:
        res = forecast_indicator_with_exog(train_df, test_df, ind)
        if res:
            all_results[ind] = res
            steps = len(res['forecast'])
            forecasted[f'{ind}_forecast'] = np.nan
            forecasted[f'{ind}_lower'] = np.nan
            forecasted[f'{ind}_upper'] = np.nan
            forecasted[f'{ind}_actual'] = np.nan
            forecasted.iloc[:steps, forecasted.columns.get_loc(f'{ind}_forecast')] = res['forecast'].values
            forecasted.iloc[:steps, forecasted.columns.get_loc(f'{ind}_lower')] = res['forecast_lower'].values
            forecasted.iloc[:steps, forecasted.columns.get_loc(f'{ind}_upper')] = res['forecast_upper'].values
            forecasted.iloc[:steps, forecasted.columns.get_loc(f'{ind}_actual')] = res['actual'].values
            if plot_results: plot_forecast_results(res, ind, train_df, test_df)

    return all_results, forecasted


In [2]:
train_selected = pd.read_csv('../../data/fix/feature_selected_recession_train.csv')
test_selected = pd.read_csv('../../data/fix/feature_selected_recession_test.csv')

In [ ]:
# Run forecasting on your selected datasets
all_results, forecasted_values = run_time_series_forecasting_selected(
    train_selected=train_selected,
    test_selected=test_selected,
    plot_results=True
)

# # Analyze performance
# performance_summary = analyze_forecast_performance(all_results)

# # Save results
# save_forecast_results(all_results, forecasted_values, 'my_forecast_results')